# Ex2 - Filtering and Sorting Data

This time we are going to pull data directly from the internet.

### Step 1. Import the necessary libraries

In [1]:
import pandas as pd
from pyspark.sql import SparkSession


In [2]:
spark = SparkSession.builder\
                    .appName('euro')\
                    .getOrCreate()

25/05/31 22:45:19 WARN Utils: Your hostname, kevin-llanos-Type1ProductConfigId resolves to a loopback address: 127.0.1.1; using 192.168.1.92 instead (on interface wlo1)
25/05/31 22:45:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/31 22:45:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Step 2. Import the dataset from this [address](https://raw.githubusercontent.com/guipsamora/pandas_exercises/master/02_Filtering_%26_Sorting/Euro12/Euro_2012_stats_TEAM.csv). 

### Step 3. Assign it to a variable called euro12.

In [3]:
import requests

url = 'https://raw.githubusercontent.com/guipsamora/pandas_exercises/master/02_Filtering_%26_Sorting/Euro12/Euro_2012_stats_TEAM.csv'

get_url = requests.get(url)


with open('data.csv', 'w', encoding='UTF-8') as f:

    f.write(get_url.text)


df = pd.read_csv('data.csv', sep=',')

euro12 = spark.read.csv('data.csv', sep=',', header=True)


25/05/31 22:45:37 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


### Step 4. Select only the Goal column.

In [4]:
euro12.printSchema()

root
 |-- Team: string (nullable = true)
 |-- Goals: string (nullable = true)
 |-- Shots on target: string (nullable = true)
 |-- Shots off target: string (nullable = true)
 |-- Shooting Accuracy: string (nullable = true)
 |-- % Goals-to-shots: string (nullable = true)
 |-- Total shots (inc. Blocked): string (nullable = true)
 |-- Hit Woodwork: string (nullable = true)
 |-- Penalty goals: string (nullable = true)
 |-- Penalties not scored: string (nullable = true)
 |-- Headed goals: string (nullable = true)
 |-- Passes: string (nullable = true)
 |-- Passes completed: string (nullable = true)
 |-- Passing Accuracy: string (nullable = true)
 |-- Touches: string (nullable = true)
 |-- Crosses: string (nullable = true)
 |-- Dribbles: string (nullable = true)
 |-- Corners Taken: string (nullable = true)
 |-- Tackles: string (nullable = true)
 |-- Clearances: string (nullable = true)
 |-- Interceptions: string (nullable = true)
 |-- Clearances off line: string (nullable = true)
 |-- Clean Sh

In [5]:
euro12.select('Goals').show()

+-----+
|Goals|
+-----+
|    4|
|    4|
|    4|
|    5|
|    3|
|   10|
|    5|
|    6|
|    2|
|    2|
|    6|
|    1|
|    5|
|   12|
|    5|
|    2|
+-----+



### Step 5. How many team participated in the Euro2012?

In [6]:
from pyspark.sql.functions import countDistinct

euro12.agg(countDistinct('Team').alias('Total_equipos')).show()

+-------------+
|Total_equipos|
+-------------+
|           16|
+-------------+



In [7]:
df['Team'].nunique()

16

### Step 6. What is the number of columns in the dataset?

In [8]:
n_cols = len(euro12.columns)

print('El numero total de columnas es de %d'%n_cols)

El numero total de columnas es de 35


In [9]:
print(f'El numero total de columnas es de {df.shape[1]}')

El numero total de columnas es de 35


### Step 7. View only the columns Team, Yellow Cards and Red Cards and assign them to a dataframe called discipline

In [10]:
discipline = euro12.select('Team', 'Yellow Cards', 'Red Cards')

discipline.show()

+-------------------+------------+---------+
|               Team|Yellow Cards|Red Cards|
+-------------------+------------+---------+
|            Croatia|           9|        0|
|     Czech Republic|           7|        0|
|            Denmark|           4|        0|
|            England|           5|        0|
|             France|           6|        0|
|            Germany|           4|        0|
|             Greece|           9|        1|
|              Italy|          16|        0|
|        Netherlands|           5|        0|
|             Poland|           7|        1|
|           Portugal|          12|        0|
|Republic of Ireland|           6|        1|
|             Russia|           6|        0|
|              Spain|          11|        0|
|             Sweden|           7|        0|
|            Ukraine|           5|        0|
+-------------------+------------+---------+



In [11]:
discipline_df = df[['Team', 'Yellow Cards', 'Red Cards']]
discipline_df

,Team,Yellow Cards,Red Cards
0,Croatia,9,0
1,Czech Republic,7,0
2,Denmark,4,0
3,England,5,0
4,France,6,0
5,Germany,4,0
6,Greece,9,1
7,Italy,16,0
8,Netherlands,5,0
9,Poland,7,1


### Step 8. Sort the teams by Red Cards, then to Yellow Cards

In [12]:
discipline.orderBy(['Yellow Cards', 'Red Cards'], ascending=[False, False]).show()

+-------------------+------------+---------+
|               Team|Yellow Cards|Red Cards|
+-------------------+------------+---------+
|             Greece|           9|        1|
|            Croatia|           9|        0|
|             Poland|           7|        1|
|     Czech Republic|           7|        0|
|             Sweden|           7|        0|
|Republic of Ireland|           6|        1|
|             France|           6|        0|
|             Russia|           6|        0|
|            England|           5|        0|
|        Netherlands|           5|        0|
|            Ukraine|           5|        0|
|            Denmark|           4|        0|
|            Germany|           4|        0|
|              Italy|          16|        0|
|           Portugal|          12|        0|
|              Spain|          11|        0|
+-------------------+------------+---------+



In [13]:
discipline_df.sort_values(['Yellow Cards', 'Red Cards'], ascending=[False, False])

,Team,Yellow Cards,Red Cards
7,Italy,16,0
10,Portugal,12,0
13,Spain,11,0
6,Greece,9,1
0,Croatia,9,0
9,Poland,7,1
1,Czech Republic,7,0
14,Sweden,7,0
11,Republic of Ireland,6,1
4,France,6,0


### Step 9. Calculate the mean Yellow Cards given per Team

In [16]:
from pyspark.sql.functions import col, avg
discipline.groupBy(col('Team')).agg(avg('Yellow Cards').alias('promedio_tarjetas_amarillas')).orderBy('promedio_tarjetas_amarillas').show()

+-------------------+---------------------------+
|               Team|promedio_tarjetas_amarillas|
+-------------------+---------------------------+
|            Germany|                        4.0|
|            Denmark|                        4.0|
|            Ukraine|                        5.0|
|            England|                        5.0|
|        Netherlands|                        5.0|
|             Russia|                        6.0|
|             France|                        6.0|
|Republic of Ireland|                        6.0|
|             Sweden|                        7.0|
|     Czech Republic|                        7.0|
|             Poland|                        7.0|
|             Greece|                        9.0|
|            Croatia|                        9.0|
|              Spain|                       11.0|
|           Portugal|                       12.0|
|              Italy|                       16.0|
+-------------------+---------------------------+


In [29]:
discipline_df.groupby('Team', as_index=False).agg(avg_amarillas= ('Yellow Cards','mean')).sort_values(by='avg_amarillas')

,Team,avg_amarillas
2,Denmark,4.0
5,Germany,4.0
3,England,5.0
8,Netherlands,5.0
15,Ukraine,5.0
12,Russia,6.0
11,Republic of Ireland,6.0
4,France,6.0
14,Sweden,7.0
9,Poland,7.0


### Step 10. Filter teams that scored more than 6 goals

### Step 11. Select the teams that start with G

### Step 12. Select the first 7 columns

### Step 13. Select all columns except the last 3.

### Step 14. Present only the Shooting Accuracy from England, Italy and Russia